# Ensembles

In [ ]:
from IPython.display import Image

import warnings
warnings.simplefilter("ignore")

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 10, 6
import seaborn as sns
%matplotlib inline

from sklearn.datasets import load_digits as load
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, BaggingRegressor
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier

import pandas as pd
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score

----------
<h1 align="center">Bagging</h1> 

** Вопросы для самоконтроля**
* Зачем нужно строить композиции над алгоритмами?
* Что такое bootstrap-выборка?
* Какие бывают варианты построения композиций N базовых алгоритмов классификации?
* Что такое Bagging?
* Почему работает Bagging?

### Bootstrap

<img src='img/bootstrap.png'>

** Важно! **
    - Бутстрепная выборка имеет такой же размер, что и исходная
    - Генерация с повторениями

### Bagging

## $$a_{Bagging}(x) = \frac{1}{M}\sum_{i=1}^M a_i(x)$$

$a_i(x)$ - обучен на бутстреп-выборке $X^i$

<img src='img/bagging.png'>

In [ ]:
iris = load()
X = iris.data
y = iris.target

f = X.shape[1]

rnd_d3 = DecisionTreeClassifier(max_features=int(f ** 0.5)) # Решающее дерево с рандомизацией в сплитах
d3 = DecisionTreeClassifier() # Обычное решающее дерево

Качество классификации одним решающим деревом:

In [ ]:
print(f"Decision tree: {cross_val_score(d3, X, y).mean():.4f}")

Качество бэггинга над решающими деревьями:

In [ ]:
print(f"Bagging: {cross_val_score(BaggingClassifier(d3), X, y).mean():.4f}")

- Какой недостаток есть у деревьев?
- Как bagging борется с этим недостатком?

- Как можно улучшить качество?
    - При построении каждого узла отбирать случайные max_features признаков и искать информативное разбиение только по одному из них.

In [ ]:
print(f"Randomized Bagging: {cross_val_score(BaggingClassifier(rnd_d3), X, y).mean():.4f}")

### Random Forest

##### Алгоритм построения случайного леса из $N$ деревьев

Для каждого $n = 1..N$:

Сгенерировать выборку $X_n$ с помощью бутстрэпа;
Построить решающее дерево $b_n$ по выборке $X_n$:
* по заданному критерию мы выбираем лучший признак, делаем разбиение в дереве по нему и так до исчерпания выборки
* дерево строится, пока в каждом листе не более $n_{min}$ объектов или пока не достигнем определенной высоты дерева
* при каждом разбиении сначала выбирается $m$ случайных признаков из $n$ исходных, и оптимальное разделение выборки ищется только среди них.

Итоговый классификатор:
    $$ a(x) = \dfrac{1}{N} \sum_{i=1}^{N} b_i(x)$$

$m$ советуют выбирать равным:
- $\sqrt{n}$ для классификации
- $\dfrac{n}{3}$ для регрессии

### Random Forest из sklearn

Полный список параметров случайного леса для задачи регрессии:

In [ ]:
"""
class sklearn.ensemble.RandomForestRegressor(
    n_estimators — число деревьев в "лесу" (по дефолту – 10)
    criterion — функция, которая измеряет качество разбиения ветки дерева (по дефолту — "mse" , так же можно выбрать "mae")
    max_features — число признаков, по которым ищется разбиение. Вы можете указать конкретное число или процент признаков, либо выбрать из доступных значений: "auto" (все признаки), "sqrt", "log2". По дефолту стоит "auto".
    max_depth — максимальная глубина дерева  (по дефолту глубина не ограничена)
    min_samples_split — минимальное количество объектов, необходимое для разделения внутреннего узла. Можно задать числом или процентом от общего числа объектов (по дефолту — 2)
    min_samples_leaf — минимальное число объектов в листе. Можно задать числом или процентом от общего числа объектов (по дефолту — 1)
    min_weight_fraction_leaf — минимальная взвешенная доля от общей суммы весов (всех входных объектов) должна быть в листе (по дефолту имеют одинаковый вес)
    max_leaf_nodes — максимальное количество листьев (по дефолту нет ограничения)
    min_impurity_split — порог для остановки наращивания дерева (по дефолту 1е-7)
    bootstrap — применять ли бустрэп для построения дерева (по дефолту True)
    oob_score — использовать ли out-of-bag объекты для оценки R^2 (по дефолту False)
    n_jobs — количество ядер для построения модели и предсказаний (по дефолту 1, если поставить -1, то будут использоваться все ядра)
    random_state — начальное значение для генерации случайных чисел (по дефолту его нет, если хотите воспроизводимые результаты, то нужно указать любое число типа int
    verbose — вывод логов по построению деревьев (по дефолту 0)
    warm_start — использует уже натренированую модель и добавляет деревьев в ансамбль (по дефолту False)
)
"""

Для задачи классификации все почти то же самое, мы приведем только те параметры, которыми RandomForestClassifier отличается от RandomForestRegressor:

In [ ]:
"""
class sklearn.ensemble.RandomForestClassifier(
    criterion — поскольку у нас теперь задача классификации, то по дефолту выбран критерий "gini" (можно выбрать "entropy")
    class_weight — вес каждого класса (по дефолту все веса равны 1, но можно передать словарь с весами, либо явно указать "balanced", тогда веса классов будут равны их исходным частям в генеральной совокупности; также можно указать "balanced_subsample", тогда веса на каждой подвыборке будут меняться в зависимости от распределения классов на этой подвыборке.
)
"""


При построениии модели в первую очередь стоит обратить внимание на следующие параметры:

- n_estimators — число деревьев в "лесу"
- criterion — критерий для разбиения выборки в вершине
- max_features — число признаков, по которым ищется разбиение
- min_samples_leaf — минимальное число объектов в листе
- max_depth — максимальная глубина дерева


### Применение RandomForest на реальной задаче (предсказание оттока клиентов)

Данные можно взять тут: https://github.com/Yorko/mlcourse_open/blob/master/data/telecom_churn.csv

In [ ]:
!wget https://raw.githubusercontent.com/Yorko/mlcourse.ai/master/data/telecom_churn.csv

In [ ]:
# Загружаем данные
df = pd.read_csv("telecom_churn.csv")

# Выбираем сначала только колонки с числовым типом данных
cols = []
for i in df.columns:
    if (df[i].dtype == "float64") or (df[i].dtype == 'int64'):
        cols.append(i)

# Разделяем на признаки и объекты
X, y = df[cols].copy(), np.asarray(df["Churn"],dtype='int8')

# Инициализируем страифицированную разбивку нашего датасета для валидации
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Инициализируем наш классификатор с дефолтными параметрами
rfc = RandomForestClassifier(random_state=42, n_jobs=-1, oob_score=True)

# Обучаем на тренировочном датасете
results = cross_val_score(rfc, X, y, cv=skf)

# Оцениваем долю верных ответов на тестовом датасете
print(f"CV accuracy score: {100 * results.mean():.2f}%")

Улучшим этот результат: посмотрим, как ведут себя кривые валидации при изменении основных параметров

In [ ]:
# Инициализируем валидацию
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
param_grid = {
    "n_estimators": [5, 10, 15, 20, 30, 50, 75, 100]
}
gscv = GridSearchCV(
    RandomForestClassifier(random_state=42, n_jobs=-1, oob_score=True),
    param_grid,
    scoring="accuracy",
    n_jobs=1,
    cv=skf,
    return_train_score=True
)
gscv.fit(X, y)

train_acc_mean = gscv.cv_results_["mean_train_score"]
test_acc_mean = gscv.cv_results_["mean_test_score"]

train_acc_std = gscv.cv_results_["std_train_score"]
test_acc_std = gscv.cv_results_["std_test_score"]

best_test_acc = gscv.best_score_
best_param_value = list(gscv.best_params_.values())[0]
print(f"Best CV accuracy is {100 * best_test_acc:.2f}%"
      f"with {best_param_value} trees")

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

trees_grid = param_grid["n_estimators"]
fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(trees_grid, train_acc_mean, alpha=0.5, color='blue', label='train')
ax.plot(trees_grid, test_acc_mean, alpha=0.5, color='red', label='cv')
ax.fill_between(trees_grid, test_acc_mean - test_acc_std, test_acc_mean + test_acc_std, color='#888888', alpha=0.4)
ax.fill_between(trees_grid, test_acc_mean - 2*test_acc_std, test_acc_mean + 2*test_acc_std, color='#888888', alpha=0.2)
ax.legend(loc='best')
ax.set_ylim([0.88,1.02])
ax.set_ylabel("Accuracy")
ax.set_xlabel("N_estimators")

Подбираем параметр max_depth:

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
param_grid = {
    "max_depth": [3, 5, 7, 9, 11, 13, 15, 17, 20, 22, 24]
}
gscv = GridSearchCV(
    RandomForestClassifier(random_state=42, n_jobs=-1, oob_score=True),
    param_grid,
    scoring="accuracy",
    n_jobs=1,
    cv=skf,
    return_train_score=True
)
gscv.fit(X, y)

train_acc_mean = gscv.cv_results_["mean_train_score"]
test_acc_mean = gscv.cv_results_["mean_test_score"]

train_acc_std = gscv.cv_results_["std_train_score"]
test_acc_std = gscv.cv_results_["std_test_score"]

best_test_acc = gscv.best_score_
best_param_value = list(gscv.best_params_.values())[0]
print(f"Best CV accuracy is {100 * best_test_acc:.2f}% "
      f"with max depth of {best_param_value} trees")

In [ ]:
trees_grid = param_grid["max_depth"]
fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(trees_grid, train_acc_mean, alpha=0.5, color='blue', label='train')
ax.plot(trees_grid, test_acc_mean, alpha=0.5, color='red', label='cv')
ax.fill_between(trees_grid, test_acc_mean - test_acc_std, test_acc_mean + test_acc_std, color='#888888', alpha=0.4)
ax.fill_between(trees_grid, test_acc_mean - 2*test_acc_std, test_acc_mean + 2*test_acc_std, color='#888888', alpha=0.2)
ax.legend(loc='best')
ax.set_ylim([0.88,1.02])
ax.set_ylabel("Accuracy")
ax.set_xlabel("Max_depth")

Параметр max_depth хорошо справляется с регуляризацией модели, и мы уже не так сильно переобучаемся. 

Параметр min_samples_leaf также выполняет функцию регуляризатора.

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
param_grid = {
    "min_samples_leaf": [1, 3, 5, 7, 9, 11, 13, 15, 17, 20, 22, 24]
}
gscv = GridSearchCV(
    RandomForestClassifier(random_state=42, n_jobs=-1, oob_score=True),
    param_grid,
    scoring="accuracy",
    n_jobs=1,
    cv=skf,
    return_train_score=True
)
gscv.fit(X, y)

train_acc_mean = gscv.cv_results_["mean_train_score"]
test_acc_mean = gscv.cv_results_["mean_test_score"]

train_acc_std = gscv.cv_results_["std_train_score"]
test_acc_std = gscv.cv_results_["std_test_score"]

best_test_acc = gscv.best_score_
best_param_value = list(gscv.best_params_.values())[0]
print(f"Best CV accuracy is {100 * best_test_acc:.2f}% "
      f"with min samples per leaf of {best_param_value} trees")

In [ ]:
trees_grid = param_grid["min_samples_leaf"]
fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(trees_grid, train_acc_mean, alpha=0.5, color='blue', label='train')
ax.plot(trees_grid, test_acc_mean, alpha=0.5, color='red', label='cv')
ax.fill_between(trees_grid, test_acc_mean - test_acc_std, test_acc_mean + test_acc_std, color='#888888', alpha=0.4)
ax.fill_between(trees_grid, test_acc_mean - 2*test_acc_std, test_acc_mean + 2*test_acc_std, color='#888888', alpha=0.2)
ax.legend(loc='best')
ax.set_ylim([0.88,1.02])
ax.set_ylabel("Accuracy")
ax.set_xlabel("Min_samples_leaf")

Рассмотрим такой параметр как max_features. Для задач классификации по умолчанию используется $\sqrt{n}$, где $n$ — число признаков.

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
param_grid = {
    "max_features": [2, 4, 6, 8, 10, 12, 14, 16]
}
gscv = GridSearchCV(
    RandomForestClassifier(random_state=42, n_jobs=-1, oob_score=True),
    param_grid,
    scoring="accuracy",
    n_jobs=1,
    cv=skf,
    return_train_score=True
)
gscv.fit(X, y)

train_acc_mean = gscv.cv_results_["mean_train_score"]
test_acc_mean = gscv.cv_results_["mean_test_score"]

train_acc_std = gscv.cv_results_["std_train_score"]
test_acc_std = gscv.cv_results_["std_test_score"]

best_test_acc = gscv.best_score_
best_param_value = list(gscv.best_params_.values())[0]
print(f"Best CV accuracy is {100 * best_test_acc:.2f}% "
      f"with max features of {best_param_value}")

In [ ]:
trees_grid = param_grid["max_features"]
fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(trees_grid, train_acc_mean, alpha=0.5, color='blue', label='train')
ax.plot(trees_grid, test_acc_mean, alpha=0.5, color='red', label='cv')
ax.fill_between(trees_grid, test_acc_mean - test_acc_std, test_acc_mean + test_acc_std, color='#888888', alpha=0.4)
ax.fill_between(trees_grid, test_acc_mean - 2*test_acc_std, test_acc_mean + 2*test_acc_std, color='#888888', alpha=0.2)
ax.legend(loc='best')
ax.set_ylim([0.88,1.02])
ax.set_ylabel("Accuracy")
ax.set_xlabel("Max_features")

В нашем случае оптимальное число признаков - 4, если судить по кросс-валидации.

Итак, итоговый перебор параметров будет выглядеть следующим образом:

In [ ]:
# Сделаем инициализацию параметров, по которым хотим сделать полный перебор
parameters = {
    "max_features": [4, 7, 10, 13],
    "min_samples_leaf": [1, 3, 5, 7],
    "max_depth": [5,10,15,20]
}
rfc = RandomForestClassifier(n_estimators=100, random_state=42, 
                             n_jobs=-1, oob_score=True, return_train_score=True)
gcv = GridSearchCV(rfc, parameters, n_jobs=-1, cv=skf, verbose=1)
gcv.fit(X, y)

### Есть ли переобучение с увеличением числа деревьев?

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
%%time

for n_estimators in [10, 40, 100, 200, 600, 1000]:
    clf = BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=n_estimators, n_jobs=4)
    clf = clf.fit(X_train, y_train)
    train_acc, test_acc = accuracy_score(clf.predict(X_train), y_train), accuracy_score(clf.predict(X_test), y_test)
    print(f"n_estimators = {n_estimators:5d}, train_acc = {train_acc:.4f} test_acc = {test_acc:.4f}")

In [ ]:
%%time

for n_estimators in [10, 40, 100, 200, 600, 1000]:
    clf = BaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth=7), n_estimators=n_estimators, n_jobs=4)
    clf = clf.fit(X_train, y_train)
    train_acc, test_acc = accuracy_score(clf.predict(X_train), y_train), accuracy_score(clf.predict(X_test), y_test)
    print(f"n_estimators = {n_estimators:5d}, train_acc = {train_acc:.4f} test_acc = {test_acc:.4f}")

In [ ]:
%%time

for n_estimators in [10, 40, 100, 200, 600, 1000]:
    clf = BaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth=14), n_estimators=n_estimators, n_jobs=4)
    clf = clf.fit(X_train, y_train)
    train_acc, test_acc = accuracy_score(clf.predict(X_train), y_train), accuracy_score(clf.predict(X_test), y_test)
    print(f"n_estimators = {n_estimators:5d}, train_acc = {train_acc:.4f} test_acc = {test_acc:.4f}")

In [ ]:
%%time

for n_estimators in [10, 40, 100, 200, 600, 1000]:
    clf = RandomForestClassifier(max_depth=14, n_estimators=n_estimators, n_jobs=4)
    clf = clf.fit(X_train, y_train)
    train_acc, test_acc = accuracy_score(clf.predict(X_train), y_train), accuracy_score(clf.predict(X_test), y_test)
    print(f"n_estimators = {n_estimators:5d}, train_acc = {train_acc:.4f} test_acc = {test_acc:.4f}")

### Out-of-bag error

<img src='img/oob.png' width=700>

**Задача** Покажите, что примерно 37% примеров остаются вне выборки бутстрэпа и не используются при построении k-го дерева.

**Решение** Пусть в выборке $l$ объектов. На каждом шаге все объекты попадают в подвыборку с возвращением равновероятно, т.е отдельный объект — с вероятностью $\dfrac{1}{l}$. Вероятность того, что объект НЕ попадет в подвыборку (т.е. его не взяли $l$ раз): $(1-\dfrac{1}{l})^l$


$$\lim_{l \rightarrow +\infty} (1-\dfrac{1}{l})^l = \dfrac{1}{e}$$

Тогда вероятность попадания конкретного объекта в подвыборку $1 - \dfrac{1}{e} \approx 63\%$.

Out-of-Bag оценка — это усредненная оценка базовых алгоритмов на тех ~37% данных, на которых они не обучались.

В случае с scikit-learn имплементацией, OOB-оценка точности (accuracy) доступна в поле класса `.oob_score_` объекта класса `RandomForestClassifier(**params)`.
OOB ошибка в таком случае будет равна `1 - clf.oob_score_`.

<h1 align="center">Выводы</h1> 

**Bagging**:
- Одна из лучших техник для построения алгоритмов ML
- Линейно уменьшает разброс и не уменьшает смещение (если не коррелированы ответы базовых алоритмов) 
- Слабое переобучение
- НО переобучение ЕСТЬ -- от сложности одного алгоритма, лучше все же немного обрезать деревья

**Random Forest**

Плюсы:
- имеет высокую точность предсказания, на большинстве задач будет лучше линейных алгоритмов; точность сравнима с точностью бустинга
- практически не чувствителен к выбросам в данных из-за случайного сэмлирования
- не чувствителен к масштабированию (и вообще к любым монотонным преобразованиям) значений признаков, связано с выбором случайных подпространств
- не требует тщательной настройки параметров, хорошо работает «из коробки». С помощью «тюнинга» параметров можно достичь прироста от 0.5 до 3% точности в зависимости от задачи и данных
- способен эффективно обрабатывать данные с большим числом признаков и классов
- одинаково хорошо обрабатывет как непрерывные, так и дискретные признаки
- редко переобучается, на практике добавление деревьев почти всегда только улучшает композицию, но на валидации, после достижения определенного количества деревьев, кривая обучения выходит на асимптоту
- хорошо работает с пропусками в данных
- предполагает возможность сбалансировать вес каждого класса на всей выборке, либо на подвыборке каждого дерева
- вычисляет близость между парами объектов, которые могут использоваться при кластеризации, обнаружении выбросов или (путем масштабирования) дают интересные представления данных
- высокая параллелизуемость и масштабируемость

Минусы:
- в отличие от одного дерева, результаты случайного леса сложнее интерпретировать
- алгоритм работает хуже многих линейных методов, когда в выборке очень много разреженных признаков (тексты, Bag of words)
- случайный лес не умеет экстраполировать, в отличие от той же линейной регрессии
- алгоритм склонен к переобучению на некоторых задачах, особенно на зашумленных данных
- для данных, включающих категориальные переменные с различным количеством уровней, случайные леса предвзяты в пользу признаков с большим количеством уровней: когда у признака много уровней, дерево будет сильнее подстраиваться именно под эти признаки, так как на них можно получить более высокое значение оптимизируемого функционала (например, прироста информации)
- больший размер получающихся моделей. Требуется $O(NK)$ памяти для хранения модели, где $K$ — число деревьев.